In [2]:
from dotenv import load_dotenv
import os
import pandas as pd
import pygsheets
import json

load_dotenv()

True

In [2]:
service_account = pygsheets.authorize(service_account_file='JSONs\\spreadsheet-automator-449222-28b7c907226f.json')

sheet = service_account.open_by_url('https://docs.google.com/spreadsheets/d/1p3ecWHthVRXvAEr900K8YOEt_c2ey98v8ym3I_GTWEA/edit?usp=sharing')


In [3]:
test = sheet.worksheet('title', 'Test')

In [4]:
from dotenv import load_dotenv
load_dotenv()

import os
import requests

API_KEY= os.environ.get('RIOT_API_KEY')

def get_puuid(summonerId=None, gameName=None, tagLine=None, region='americas'):

    if summonerId is not None:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = 'lol/summoner/v4/summoners/'
        response = requests.get(root_url+endpoint+summonerId+'?api_key='+API_KEY)
        return response.json()['puuid']
    else:
        root_url = f'https://{region}.api.riotgames.com/'
        endpoint = f'riot/account/v1/accounts/by-riot-id/{gameName}/{tagLine}'
        response = requests.get(root_url+endpoint+'?api_key='+API_KEY)
        return response.json()['puuid']

def get_idtag_from_puuid(puuid=None):
    root_url = 'https://americas.api.riotgames.com/'
    endpoint = 'riot/account/v1/accounts/by-puuid/'

    response = requests.get(root_url+endpoint+puuid+'?api_key'+API_KEY)

    id = {
        'gameName' : response.json()['gameName'],
        'tagLine' : response.json()['tagLine']
    }

    return id

In [5]:
def get_ladder(top=None,):
    root = 'https://na1.api.riotgames.com/'
    chall = 'lol/league/v4/challengerleagues/by-queue/RANKED_SOLO_5x5'
    gm = 'lol/league/v4/grandmasterleagues/by-queue/RANKED_SOLO_5x5'
    masters = 'lol/league/v4/masterleagues/by-queue/RANKED_SOLO_5x5'
    chall_response = requests.get(root + chall + '?api_key=' + API_KEY)
    gm_response = requests.get(root + gm + '?api_key=' + API_KEY)
    masters_response = requests.get(root + masters + '?api_key=' + API_KEY)
    chall_df = pd.DataFrame(chall_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    gm_df = pd.DataFrame(gm_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)
    masters_df = pd.DataFrame(masters_response.json()['entries']).sort_values('leaguePoints', ascending=False).reset_index(drop=True)

    ladder = pd.concat([chall_df, gm_df, masters_df]).reset_index(drop=True)[0:top]
    ladder = ladder.drop(columns='rank').reset_index(drop=False).rename(columns={'index':'rank'})
    ladder['rank'] = ladder['rank'] + 1
    
    return ladder

def get_match_history(region=None, puuid=None, start=0, count=20):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'lol/match/v5/matches/by-puuid/{puuid}/ids'
    query_params = f'?start={start}&count={count}'

    response = requests.get(root_url + endpoint + query_params +'&api_key=' + API_KEY)


    return response.json()

def get_match_data_from_id(region=None, matchId=None):
    root_url = f'https://{region}.api.riotgames.com/'
    endpoint = f'/lol/match/v5/matches/{matchId}'

    response = requests.get(root_url+endpoint+'?api_key='+API_KEY)

    return response.json()

In [6]:
get_ladder(300)

,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1,yKKXTvcFPlTL8qIFO8j7h5pIsihllh92VliC1m1IlO7IFcM,1603,124,83,True,False,False,True
1,2,U0NRKUez6uuiYSLieKIlT_LL25pd-vaA0kjdsIMMJLvjsZ8,1555,98,57,False,False,False,False
2,3,zTDVfcGmuxQnHL5PIc0dfCIe6t_nAVgWMJOR0dCispnn1Zyg,1424,82,48,False,False,False,False
3,4,jbMwvSqQeMSQaNYp8WTTIvElc1PLv2e61ied3LcI0kSF4Mg,1413,101,72,True,False,False,True
4,5,CGJvSeU4ZQWpJnxUL9I0h9RCAsqFpl5kUHbLGluse9XLQ-...,1369,83,43,False,False,False,False
...,...,...,...,...,...,...,...,...,...
295,296,AK4inIx7EIXFV73oerY60BuHHPv5gwhOenOxkb9fFH-OOkw,471,56,47,False,False,True,False
296,297,Vu_6ftY4n8get971XobsxmHnD0eKwlED36nxIt1YEPPxv40,470,59,45,False,False,True,False
297,298,CA3BiOsAceCLfDBu_xbpjr-WO9GqVByNT01mBZLLvZn0vW4,469,65,47,False,False,False,True
298,299,Lz7IFUCJZSZPhOPkGyl4YB3Ac-dMrBKE4h-LagvH-9I1uNQ,468,75,68,False,False,True,False


In [7]:
get_puuid(gameName="denial", tagLine = "viego")

'podsNUGPSvp7NFmwK5CN2b7-kuJiDeciSCveMXtuAM8K8qYkGt71g815t0R6x5LHgpTgtpSuFs1cBQ'

In [8]:
get_match_history(region="americas", puuid="5kXhUQSJDvpYKmmsKN5OzSvDhel0y2YxGrgCMsl_CabC_8NXPZTCJTi5KYWSxa62sIrPEVHaQ8FVyw")

['NA1_5216854883',
 'NA1_5216832915',
 'NA1_5215873552',
 'NA1_5215859385',
 'NA1_5215844362',
 'NA1_5215824821',
 'NA1_5215806452',
 'NA1_5215787352',
 'NA1_5214993103',
 'NA1_5214977325',
 'NA1_5214956912',
 'NA1_5214933780',
 'NA1_5214918120',
 'NA1_5214880242',
 'NA1_5214862616',
 'NA1_5214850122',
 'NA1_5214816057',
 'NA1_5212746743',
 'NA1_5212728948',
 'NA1_5211161275']

In [9]:
get_match_data_from_id(region="americas", matchId="NA1_5216854883")

{'metadata': {'dataVersion': '2',
  'matchId': 'NA1_5216854883',
  'participants': ['CAQwA0cxLSoWy8AnZV_OZFqewvkiUaKwIKK8H_DDYeGyiCqY-2iLqMNp4RIVEd3wFMXXj4WHB3V-Pw',
   'CNTIkiH3gX88285_Uq2bzYjT4P5Cb6s2-xVzGucgqvR_FHowHt-uua40jJNSLqGpTkiV3lk2h7JRAQ',
   '5kXhUQSJDvpYKmmsKN5OzSvDhel0y2YxGrgCMsl_CabC_8NXPZTCJTi5KYWSxa62sIrPEVHaQ8FVyw',
   'XQbgZrSaGzaT8DU7DNIi_HxqyA0C2bO2kvLEyPWU_MyzEtZFxUjoA11xZX8_piBeTKfgtlBimeOESA',
   'GdMw1iwqCT6oeLtOjhetwDOYRNERYvx4pDTyT_ewZ7OtNhLsx6XXFW9ISz6lHHunZWXL5hV5PY9Uug',
   'JvdIq_XtbVHaHHmTOMb8fhYyNoyIC77Ixu_qiwNs864o95lksOM4mVzVMKNZ5QpExhmMPUj9tRiuGw',
   'RWq385j9mCyUV6pVriv6nTEl4b_laYcfDCvbS2RlnPhRe0YPtLxdpWOXQioGu9WHH37hr8VgnRvGow',
   'fkdLFz0qHueq-aMYpaY7htIOauTZYdzGkJXJ6J-oFzOXxi9FZUeZnt2l-DTi-Y9mhofQR9CxFxLsRA',
   'BWD7QjzqzTbawGUAFhVhyYS_GX8puz4mCJ-9NkRfoj12uGI2uaHWgKNLXKYScIlHD8l-SCmiY9Ppdw',
   'qVWlIzEdvjtm-gLPAcuJUkLHLPg7YqWLranvhG7judrkQLbeTKlhao6IsW61Q2sCE05XwiquL7iP6g']},
 'info': {'endOfGameResult': 'GameComplete',
  'gameCreation': 1738

In [13]:
matches = get_match_history(region="americas", puuid="5kXhUQSJDvpYKmmsKN5OzSvDhel0y2YxGrgCMsl_CabC_8NXPZTCJTi5KYWSxa62sIrPEVHaQ8FVyw", count=100)
with open('snowballs.json', 'r') as file:
    snowballs = json.load(file)
friendsList = {'denial#viego', '23ejd#NA1', 'NightIsShadow#NA1', 'demon slayer yi#NA1', 'Alias Mudd#NA1', 'Azirs Pantaloons#Azir', 'Kiramdzz#1204', 'OriKitty#meow'}
flag = False
for x in matches:
    if flag:
        break
    match_data = get_match_data_from_id(region = 'americas', matchId = x)
    if match_data["info"]["gameMode"] == 'ARAM':
        for player in match_data["info"]["participants"]:
            name = player["riotIdGameName"] + '#' + player["riotIdTagline"]
            #if name in friendsList:
            if name not in snowballs:
                puuid = player['puuid']
                url = f'https://na1.api.riotgames.com/lol/challenges/v1/player-data/{puuid}?api_key={API_KEY}'
                challengeData = (requests.get(url))
                if challengeData.status_code == 200:
                    challengeData = (challengeData.json())['challenges']
                    challenge_id_to_find = 101203
                    value = None
                    for index, challenge in enumerate(challengeData):
                        if challenge["challengeId"] == challenge_id_to_find:
                            value = challenge["value"]
                            break
                    if value is not None:
                        snowballs[name] = value
                else:
                    print(challengeData.status_code)
                    flag = True
                    break
with open('snowballs.json', 'w') as file:
    json.dump(snowballs, file)
print(snowballs)


503
{'GrandpaROOT#NA1': 17799.0, 'WAWA#CATT': 4058.0, 'NightIsShadow#NA1': 13144.0, 'Kiramdzz#1204': 876.0, 'demon slayer yi#NA1': 2527.0, 'Phaeda#NA1': 6360.0, 'redangeL265#NA369': 2886.0, 'This Guy stinks#123': 2218.0, 'wow#na12': 2824.0, 'Firmly grasp it#NA1': 3363.0, 'BloodSpatter#NA1': 2265.0, 'TheWilliam#NA0': 20117.0, 'Teshkey#NA1': 32906.0, 'Quappy#NA1': 2755.0, 'Moemeomeow#NA1': 4614.0, 'HHHHHHAOMIN#NA1': 8043.0, 'PKThundahCat#NA1': 10172.0, 'kirocatt#NA1': 21691.0, 'сum diff#arams': 22737.0, 'saIakutra#3128': 5199.0, 'Umbrasong#NA1': 4853.0, 'Octapus#1006': 8837.0, 'GenericDuCouteau#NA1': 11796.0, 'bleach op 6#NA1': 16609.0, 'nickgurenko9#3560': 6667.0, 'ninjafamily#ninja': 2274.0, 'straight4free#NA1': 6213.0, 'OriKitty#meow': 1384.0, 'FrankWithYou#frank': 696.0, 'Lavender#deino': 197.0, 'hasumi#keito': 5067.0, 'Dirt#7137': 15243.0, 'SushiboiBrazil#NA1': 3342.0, 'Elegance#nsfw': 4414.0, 'ZaMonster#2740': 6598.0, 'str0mb0lli#NA1': 984.0, 'THC Hisoka#NA1': 9553.0, 'LOL Teempo#N